
Facial Recognition using PyTorch and OpenCV

https://ritik12.medium.com/facial-recognition-using-pytorch-and-opencv-467c4e41d1f


Machine Learning - Face Recognition CNN Pytorch.ipynb
https://github.com/rubencg195/Pytorch-Tutorials/blob/master/Machine%20Learning%20-%20Face%20Recognition%20CNN%20Pytorch.ipynb



Face Recognition Using Pytorch
https://github.com/timesler/facenet-pytorch

Face Landmarks Detection With PyTorch

https://towardsdatascience.com/face-landmarks-detection-with-pytorch-4b4852f5e9c4



다중입력 deep neural network
https://rosenfelder.ai/multi-input-neural-network-pytorch/



Understanding dimensions in PyTorch
https://towardsdatascience.com/understanding-dimensions-in-pytorch-6edf9972d3be

학습하기 
https://github.com/deeplearningzerotoall/PyTorch

In [1]:
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import ToTensor

from FaceFeatureDataset import FaceFeatureDataset

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cuda device


In [2]:
batch_size = 10
epochs = 10

In [3]:
#linear_layer_2d = nn.Linear(in_features=68, out_features=128)
#flatten = nn.Flatten()
#input_2d = torch.randn(2, 2, 68) # n, channel, features
#print(input_2d)
#print(input_2d.shape)
#output_2d = linear_layer_2d(input_2d)
#print('output_2d' , output_2d)
#print(output_2d.size())

#output_1d = flatten(input_2d)
#print(output_1d.size())
#linear_layer_2d(output_1d)
#print("1D ",  output_1d)

#test_sequence = nn.Sequential(nn.Linear(64, 32), nn.ReLU(), nn.Linear(32,10))
#out = test_sequence(input_2d)
#print(out)

In [4]:
'''
testload = pd.read_csv("./outimg/Train/facefeature.csv")
landmarks = np.array(testload.iloc[0, 1:])
landmarks = landmarks.astype('float').reshape(-1, 2)
print(landmarks)

dataiter = iter(testload)
landmark = next(dataiter)
print(landmark)
# print(landmarks.shape)
'''

'\ntestload = pd.read_csv("./outimg/Train/facefeature.csv")\nlandmarks = np.array(testload.iloc[0, 1:])\nlandmarks = landmarks.astype(\'float\').reshape(-1, 2)\nprint(landmarks)\n\ndataiter = iter(testload)\nlandmark = next(dataiter)\nprint(landmark)\n# print(landmarks.shape)\n'

In [5]:
# Create DataLoader 

training_data = FaceFeatureDataset(feature_file="./outimg/Train/facefeature.csv", label_file="./Dataset/Train/csv/train.csv")
test_data = FaceFeatureDataset(feature_file="./outimg/Test/facefeature.csv", label_file="./Dataset/Test/csv/test.csv")

train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# 데이터 로드 확인 
for X, y in train_loader:
    print(f"Shape of X [N, F, C]: {X.shape}") # N , Channel, H= width W = height
    print(f"Shape of Tensor y: {y.shape} {y.dtype}")       
    break

n_total_steps = len(train_loader)
# print(f'Traing dat length {n_total_steps}')
# print(y)
print('X type', X.dtype)
print('y type', y.dtype)


Shape of X [N, F, C]: torch.Size([10, 136])
Shape of Tensor y: torch.Size([10, 33]) torch.float32
X type torch.float32
y type torch.float32


In [6]:
#
#dataiter = iter(train_loader)
#landmark, labels = next(dataiter)
#print("landmark Shape ", landmark.shape)
#print(landmark)
#flatten = nn.Flatten()
#linear1 = nn.Linear(68, 32)
#x = flatten(landmark)
#print(x)
#print(x.size())
# print(landmark.shape)
# print('linear1', x.shape)


In [7]:

num_classes = 33

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        #self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(68 * 2, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
            #nn.ReLU(),
        )
    

    def forward(self, x):
        # x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=136, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=33, bias=True)
  )
)


In [8]:
#criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        # print(pred)
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print('batch',  batch)
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
# Training
epochs = 1000

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, criterion, optimizer)    
print("Done!")

Epoch 1
-------------------------------
loss: 0.291628  [    0/  100]
Epoch 2
-------------------------------
loss: 0.313901  [    0/  100]
Epoch 3
-------------------------------
loss: 0.298137  [    0/  100]
Epoch 4
-------------------------------
loss: 0.296619  [    0/  100]
Epoch 5
-------------------------------
loss: 0.283954  [    0/  100]
Epoch 6
-------------------------------
loss: 0.280179  [    0/  100]
Epoch 7
-------------------------------
loss: 0.256994  [    0/  100]
Epoch 8
-------------------------------
loss: 0.276223  [    0/  100]
Epoch 9
-------------------------------
loss: 0.242109  [    0/  100]
Epoch 10
-------------------------------
loss: 0.247070  [    0/  100]
Epoch 11
-------------------------------
loss: 0.252230  [    0/  100]
Epoch 12
-------------------------------
loss: 0.265435  [    0/  100]
Epoch 13
-------------------------------
loss: 0.245964  [    0/  100]
Epoch 14
-------------------------------
loss: 0.233807  [    0/  100]
Epoch 15
------

In [11]:
PATH = './cifar_net.pth'
torch.save(model.state_dict(), PATH)

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    print('test size', size )
    # num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)            
            print('pred =', pred)
            #print('loss', loss)
            #print('real', y)



In [13]:

test(test_loader, model, criterion)

print("Done !")

test size 100
pred = tensor([[0.4909, 0.4836, 0.4938, 0.5097, 0.5193, 0.5208, 0.4750, 0.5006, 0.5206,
         0.5122, 0.5197, 0.5000, 0.4934, 0.5133, 0.4834, 0.4903, 0.5119, 0.5337,
         0.4948, 0.5207, 0.4975, 0.5044, 0.4972, 0.4983, 0.5190, 0.4885, 0.5229,
         0.4886, 0.4809, 0.5040, 0.5234, 0.5174, 0.4837],
        [0.4908, 0.4867, 0.4944, 0.5116, 0.5191, 0.5232, 0.4762, 0.5019, 0.5229,
         0.5119, 0.5218, 0.4995, 0.4953, 0.5160, 0.4829, 0.4913, 0.5111, 0.5358,
         0.4952, 0.5227, 0.4998, 0.5080, 0.4984, 0.4991, 0.5198, 0.4897, 0.5274,
         0.4905, 0.4827, 0.5059, 0.5259, 0.5187, 0.4865],
        [0.4932, 0.4857, 0.4957, 0.5116, 0.5223, 0.5225, 0.4761, 0.5037, 0.5238,
         0.5135, 0.5208, 0.5010, 0.4949, 0.5163, 0.4863, 0.4936, 0.5138, 0.5373,
         0.4978, 0.5237, 0.4985, 0.5052, 0.4985, 0.4997, 0.5213, 0.4896, 0.5257,
         0.4901, 0.4829, 0.5062, 0.5253, 0.5201, 0.4857],
        [0.4919, 0.4863, 0.4964, 0.5122, 0.5206, 0.5237, 0.4754, 0.5015, 0.5